In [ ]:
#gpu 사용 가능 확인용
import torch
torch.cuda.is_available()

True

In [ ]:
# 1. Cifar-10 이미지 데이터셋을 다운받고 분류하는 코드를 구현하세요.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torchsummary import summary
import torch.nn.functional as F

# 데이터 전처리 및 데이터 로드
transform = transforms.Compose([
    transforms.ToTensor()
])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# CIFAR-10 데이터셋 로드
cifar_train = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

# 데이터셋으로 DataLoader 생성
batch_size = 64

train_loader = DataLoader(cifar_train, batch_size=batch_size, shuffle=True)

''' Image 확인용
import matplotlib.pyplot as plt
import numpy as np
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()
dataiter = iter(train_loader)
images, labels = next(dataiter)

# show images
imshow(torchvision.utils.make_grid(images))
'''

#신경망 정의
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.conv3 = nn.Conv2d(64, 128, 3)
        self.drop1 = nn.Dropout()
        self.fc2 = nn.Linear(512 * 2 * 2, 128)
        self.fc3 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)

        x = F.relu(self.conv2(x))
        x = self.pool(x)

        x = F.relu(self.conv3(x))

        x = torch.flatten(x, 1)
        x = F.relu(self.fc2(x))
        x = self.drop1(x)
        x = self.fc3(x)
        return x

# 모델, 손실 함수, 최적화 함수 정의
model = SimpleCNN().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
summary(model, input_size=(3, 32, 32)) # Summary 출력

# 훈련
epochs = 50
for epoch in range(epochs):
    correct = 0
    total = 0
    for imgs, labels in train_loader:
        optimizer.zero_grad()
        imgs = imgs.to(device)
        outputs = model(imgs)
        _, prediction = torch.max(outputs.data, 1)
        new_labels = [[0.0 for tt in range(10)] for t in range(len(labels))] # 0 0 0 1 0 0 0 0 0 0 형태로 만들어주기 위한 배열
        for t in range(len(labels)):
          new_labels[t][labels[t]] = 1.0
        new_labels = torch.tensor(new_labels)
        labels = labels.to(device)
        new_labels = new_labels.to(device)
        prediction = prediction.to(device)
        loss = criterion(outputs, new_labels) #loss 계산(BCEWithLogitsLoss)
        loss.backward()
        optimizer.step()
        total += (prediction == prediction).sum().item()
        correct += (prediction == labels).sum().item()

    print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}, Accuracy: {correct / total * 100:.2f}%') # Loss, Accuracy 출력

print('훈련 완료')

100%|██████████| 170498071/170498071 [00:12<00:00, 13686070.31it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 30, 30]             896
         MaxPool2d-2           [-1, 32, 15, 15]               0
            Conv2d-3           [-1, 64, 13, 13]          18,496
         MaxPool2d-4             [-1, 64, 6, 6]               0
            Conv2d-5            [-1, 128, 4, 4]          73,856
            Linear-6                  [-1, 128]         262,272
           Dropout-7                  [-1, 128]               0
            Linear-8                   [-1, 10]           1,290
Total params: 356,810
Trainable params: 356,810
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.39
Params size (MB): 1.36
Estimated Total Size (MB): 1.77
--------------------------------------------------------

In [ ]:
# 테스트
model.eval().to(device)
correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in train_loader:
        imgs = imgs.to(device)
        outputs = model(imgs)
        _, prediction = torch.max(outputs.data, 1)
        new_labels = [[0.0 for tt in range(10)] for t in range(len(labels))]
        for t in range(len(labels)):
          new_labels[t][labels[t]] = 1.0
        new_labels = torch.tensor(new_labels)
        new_labels = new_labels.to(device)
        labels = labels.to(device)
        prediction = prediction.to(device)
        loss = criterion(outputs, new_labels)
        total += (prediction == prediction).sum().item()
        correct += (prediction == labels).sum().item()
        print(f'Loss: {loss.item():.4f} Accuracy: {(correct / total * 100):.2f}%') # Loss, Accuracy

accuracy = correct / total
print(f'Final Test Accuracy: {accuracy * 100:.2f}%') # Accuracy

Loss: 0.0404 Accuracy: 92.19%
Loss: 0.0298 Accuracy: 93.75%
Loss: 0.0331 Accuracy: 93.75%
Loss: 0.0223 Accuracy: 94.53%
Loss: 0.0221 Accuracy: 95.31%
Loss: 0.0291 Accuracy: 95.57%
Loss: 0.0516 Accuracy: 94.87%
Loss: 0.0335 Accuracy: 94.73%
Loss: 0.0204 Accuracy: 95.14%
Loss: 0.0451 Accuracy: 95.00%
Loss: 0.0379 Accuracy: 95.17%
Loss: 0.0327 Accuracy: 94.92%
Loss: 0.0397 Accuracy: 94.95%
Loss: 0.0272 Accuracy: 95.20%
Loss: 0.0310 Accuracy: 95.31%
Loss: 0.0215 Accuracy: 95.51%
Loss: 0.0215 Accuracy: 95.68%
Loss: 0.0350 Accuracy: 95.49%
Loss: 0.0338 Accuracy: 95.31%
Loss: 0.0283 Accuracy: 95.31%
Loss: 0.0469 Accuracy: 95.16%
Loss: 0.0314 Accuracy: 95.10%
Loss: 0.0342 Accuracy: 95.11%
Loss: 0.0270 Accuracy: 95.05%
Loss: 0.0439 Accuracy: 94.94%
Loss: 0.0335 Accuracy: 95.01%
Loss: 0.0272 Accuracy: 94.97%
Loss: 0.0230 Accuracy: 95.09%
Loss: 0.0169 Accuracy: 95.20%
Loss: 0.0154 Accuracy: 95.26%
Loss: 0.0299 Accuracy: 95.26%
Loss: 0.0329 Accuracy: 95.21%
Loss: 0.0152 Accuracy: 95.27%
Loss: 0.01

In [ ]:
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
batch_size = 64
test_loader = DataLoader(testset, batch_size=batch_size, shuffle=True)

correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in test_loader:
        imgs = imgs.to(device)
        outputs = model(imgs)
        _, prediction = torch.max(outputs.data, 1)
        new_labels = [[0.0 for tt in range(10)] for t in range(len(labels))]
        for t in range(len(labels)):
          new_labels[t][labels[t]] = 1.0
        new_labels = torch.tensor(new_labels)
        new_labels = new_labels.to(device)
        labels = labels.to(device)
        prediction = prediction.to(device)
        loss = criterion(outputs, new_labels)
        total += (prediction == prediction).sum().item()
        correct += (prediction == labels).sum().item()
        print(f'Loss: {loss.item():.4f} Accuracy: {(correct / total * 100):.2f}%') # Loss, Accuracy

accuracy = correct / total
print(f'Final Test Accuracy: {accuracy * 100:.2f}%') # Accuracy

Files already downloaded and verified
Loss: 0.1725 Accuracy: 73.44%
Loss: 0.2176 Accuracy: 71.88%
Loss: 0.1776 Accuracy: 72.92%
Loss: 0.1896 Accuracy: 73.44%
Loss: 0.1818 Accuracy: 73.12%
Loss: 0.1167 Accuracy: 73.44%
Loss: 0.2132 Accuracy: 72.32%
Loss: 0.2505 Accuracy: 72.07%
Loss: 0.2348 Accuracy: 72.05%
Loss: 0.0982 Accuracy: 73.28%
Loss: 0.2489 Accuracy: 73.01%
Loss: 0.1576 Accuracy: 73.18%
Loss: 0.1467 Accuracy: 73.08%
Loss: 0.1901 Accuracy: 72.99%
Loss: 0.1669 Accuracy: 73.23%
Loss: 0.1958 Accuracy: 72.75%
Loss: 0.2499 Accuracy: 72.43%
Loss: 0.1776 Accuracy: 71.79%
Loss: 0.2677 Accuracy: 71.79%
Loss: 0.1864 Accuracy: 71.88%
Loss: 0.1347 Accuracy: 72.25%
Loss: 0.2490 Accuracy: 72.16%
Loss: 0.2556 Accuracy: 72.15%
Loss: 0.2392 Accuracy: 71.88%
Loss: 0.1305 Accuracy: 71.94%
Loss: 0.1455 Accuracy: 72.36%
Loss: 0.2197 Accuracy: 72.34%
Loss: 0.2406 Accuracy: 72.43%
Loss: 0.0925 Accuracy: 72.90%
Loss: 0.2066 Accuracy: 72.97%
Loss: 0.3833 Accuracy: 72.38%
Loss: 0.1535 Accuracy: 72.56%
Lo

In [ ]:
#2. Cifar-10 이미지를 아래의 이미지처럼 4개씩 묶고 분류할 수 있는 모델을 구현하세요.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torchsummary import summary
import torch.nn.functional as F

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

# 데이터 전처리 및 데이터 로드
transform = transforms.Compose([
    transforms.ToTensor(),
    normalize
])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# CIFAR-10 데이터셋 로드
cifar_train = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

class CIFAR10Dataset(Dataset):
    def __init__(self, original_dataset):
        self.original_dataset = original_dataset

    def __len__(self):
        return len(self.original_dataset) // 4  # 4개씩 묶어서 사용

    def __getitem__(self, idx):
        img1, label1 = self.original_dataset[idx * 4]
        img2, label2 = self.original_dataset[idx * 4 + 1]
        img3, label3 = self.original_dataset[idx * 4 + 2]
        img4, label4 = self.original_dataset[idx * 4 + 3]

        concatenated_img1 = torch.cat([img1, img2], dim=2)
        concatenated_img2 = torch.cat([img3, img4], dim=2)
        concatenated_img = torch.cat([concatenated_img1, concatenated_img2], dim=1)
        concatenated_lbl = torch.tensor([label1, label2, label3, label4])

        return concatenated_img, concatenated_lbl

# 이어붙인 데이터셋으로 DataLoader 생성
batch_size = 64

concatenated_train_data = CIFAR10Dataset(cifar_train)
train_loader = DataLoader(concatenated_train_data, batch_size=batch_size, shuffle=True)

''' Image 확인용
import matplotlib.pyplot as plt
import numpy as np
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()
dataiter = iter(train_loader)
images, labels = next(dataiter)

# show images
imshow(torchvision.utils.make_grid(images))
'''

#신경망 정의
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.conv3 = nn.Conv2d(64, 128, 3)
        self.conv4 = nn.Conv2d(128, 256, 3)
        self.drop1 = nn.Dropout()
        self.fc1 = nn.Linear(256 * 2 * 2, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)

        x = F.relu(self.conv2(x))
        x = self.pool(x)

        x = F.relu(self.conv3(x))
        x = self.pool(x)

        x = F.relu(self.conv4(x))
        x = self.pool(x)

        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.drop1(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# 모델, 손실 함수, 최적화 함수 정의
model = SimpleCNN().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
summary(model, input_size=(3, 64, 64)) # Summary 출력

# 훈련
epochs = 50
for epoch in range(epochs):
    correct = 0
    total = 0
    for imgs, labels in train_loader:
        optimizer.zero_grad()
        imgs = imgs.to(device)
        outputs = model(imgs)
        new_labels = [[0.0 for tt in range(10)] for t in range(len(labels))] # 1 0 0 1 0 0 1 1 0 0 형태로 만들어주기 위한 배열
        for t in range(len(labels)):
          for j in range(len(labels[t])):
            new_labels[t][labels[t][j]] = 1.0
        labels = torch.tensor(new_labels)
        labels = labels.to(device)
        loss = criterion(outputs, labels) #loss 계산(BCEWithLogitsLoss)
        loss.backward()
        optimizer.step()
        pred_soft = torch.sigmoid(outputs)
        pred_binary = pred_soft.gt(0.5).type(outputs.type())
        total += (labels == labels).sum().item()
        correct += (pred_binary == labels).sum().item()

    print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}, Accuracy: {correct / total * 100:.2f}%') # Loss, Accuracy 출력

print('훈련 완료')

Files already downloaded and verified
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 62, 62]             896
         MaxPool2d-2           [-1, 32, 31, 31]               0
            Conv2d-3           [-1, 64, 29, 29]          18,496
         MaxPool2d-4           [-1, 64, 14, 14]               0
            Conv2d-5          [-1, 128, 12, 12]          73,856
         MaxPool2d-6            [-1, 128, 6, 6]               0
            Conv2d-7            [-1, 256, 4, 4]         295,168
         MaxPool2d-8            [-1, 256, 2, 2]               0
            Linear-9                  [-1, 512]         524,800
          Dropout-10                  [-1, 512]               0
           Linear-11                  [-1, 128]          65,664
           Linear-12                   [-1, 10]           1,290
Total params: 980,170
Trainable params: 980,170
Non-trainable par

In [ ]:
# 테스트
model.eval().to(device)
correct = 0
total = 0
correct2 = 0
total2 = 0

with torch.no_grad():
    for imgs, labels in train_loader:
        imgs = imgs.to(device)
        outputs = model(imgs)
        pred_soft = torch.sigmoid(outputs)
        pred_binary = pred_soft.gt(0.5).type(outputs.type())
        new_labels = [[0.0 for tt in range(10)] for t in range(len(labels))]
        for t in range(len(labels)):
          for j in range(len(labels[t])):
            new_labels[t][labels[t][j]] = 1.0
        labels = torch.tensor(new_labels)
        labels = labels.to(device)
        loss = criterion(outputs, labels)
        total2 += (labels == labels).sum().item()
        correct2 += (pred_binary == labels).sum().item()
        for t in range(len(labels)):
          for j in range(len(labels[t])):
            if (labels[t][j] == 1.0):
              total += 1
              if (pred_binary[t][j] == 1.0):
                correct += 1
        print(f'Loss: {loss.item():.4f} Accuracy: {(correct / total * 100):.2f}%') # Label이 1일 때, 추정값도 1인 개수 비율
        print(f'Loss: {loss.item():.4f} Accuracy2: {(correct2 / total2 * 100):.2f}%') # 추정값 중 얼마나 맞았는지에 대한 비율 (Label과 추정값이 얼마나 같은지에 대한 비율)

accuracy = correct / total
accuracy2 = correct2 / total2
print(f'Final Test Accuracy: {accuracy * 100:.2f}%') # Label이 1일 때, 추정값도 1인 개수 비율
print(f'Final Test Accuracy2: {accuracy2 * 100:.2f}%') # 추정값 중 얼마나 맞았는지에 대한 비율 (Label과 추정값이 얼마나 같은지에 대한 비율)

Loss: 0.0254 Accuracy: 98.22%
Loss: 0.0254 Accuracy2: 99.06%
Loss: 0.0213 Accuracy: 98.68%
Loss: 0.0213 Accuracy2: 99.38%
Loss: 0.0151 Accuracy: 98.81%
Loss: 0.0151 Accuracy2: 99.48%
Loss: 0.0195 Accuracy: 98.54%
Loss: 0.0195 Accuracy2: 99.34%
Loss: 0.0125 Accuracy: 98.57%
Loss: 0.0125 Accuracy2: 99.38%
Loss: 0.0303 Accuracy: 98.73%
Loss: 0.0303 Accuracy2: 99.40%
Loss: 0.0140 Accuracy: 98.78%
Loss: 0.0140 Accuracy2: 99.40%
Loss: 0.0210 Accuracy: 98.82%
Loss: 0.0210 Accuracy2: 99.41%
Loss: 0.0089 Accuracy: 98.90%
Loss: 0.0089 Accuracy2: 99.44%
Loss: 0.0147 Accuracy: 98.96%
Loss: 0.0147 Accuracy2: 99.44%
Loss: 0.0311 Accuracy: 98.93%
Loss: 0.0311 Accuracy2: 99.39%
Loss: 0.0219 Accuracy: 98.88%
Loss: 0.0219 Accuracy2: 99.38%
Loss: 0.0316 Accuracy: 98.79%
Loss: 0.0316 Accuracy2: 99.34%
Loss: 0.0153 Accuracy: 98.81%
Loss: 0.0153 Accuracy2: 99.35%
Loss: 0.0243 Accuracy: 98.80%
Loss: 0.0243 Accuracy2: 99.35%
Loss: 0.0220 Accuracy: 98.84%
Loss: 0.0220 Accuracy2: 99.35%
Loss: 0.0216 Accuracy: 9

In [ ]:
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

concatenated_test_data = CIFAR10Dataset(testset)
# 이어붙인 데이터셋으로 DataLoader 생성
batch_size = 64
test_loader = DataLoader(concatenated_test_data, batch_size=batch_size, shuffle=True)

correct = 0
total = 0
correct2 = 0
total2 = 0

with torch.no_grad():
    for imgs, labels in test_loader:
        imgs = imgs.to(device)
        outputs = model(imgs)
        pred_soft = torch.sigmoid(outputs)
        pred_binary = pred_soft.gt(0.5).type(outputs.type())
        new_labels = [[0.0 for tt in range(10)] for t in range(len(labels))]
        for t in range(len(labels)):
          for j in range(len(labels[t])):
            new_labels[t][labels[t][j]] = 1.0
        labels = torch.tensor(new_labels)
        labels = labels.to(device)
        loss = criterion(outputs, labels)
        total2 += (labels == labels).sum().item()
        correct2 += (pred_binary == labels).sum().item()
        for t in range(len(labels)):
          for j in range(len(labels[t])):
            if (labels[t][j] == 1.0):
              total += 1
              if (pred_binary[t][j] == 1.0):
                correct += 1
        print(f'Loss: {loss.item():.4f} Accuracy: {(correct / total * 100):.2f}%') # Label이 1일 때, 추정값도 1인 개수 비율
        print(f'Loss: {loss.item():.4f} Accuracy2: {(correct2 / total2 * 100):.2f}%') # 추정값 중 얼마나 맞았는지에 대한 비율 (Label과 추정값이 얼마나 같은지에 대한 비율)

accuracy = correct / total
accuracy2 = correct2 / total2
print(f'Final Test Accuracy: {accuracy * 100:.2f}%') # Label이 1일 때, 추정값도 1인 개수 비율
print(f'Final Test Accuracy2: {accuracy2 * 100:.2f}%') # 추정값 중 얼마나 맞았는지에 대한 비율 (Label과 추정값이 얼마나 같은지에 대한 비율)

Files already downloaded and verified
Loss: 1.2582 Accuracy: 61.19%
Loss: 1.2582 Accuracy2: 75.78%
Loss: 1.5690 Accuracy: 62.12%
Loss: 1.5690 Accuracy2: 75.55%
Loss: 1.2715 Accuracy: 63.00%
Loss: 1.2715 Accuracy2: 76.04%
Loss: 1.3139 Accuracy: 62.83%
Loss: 1.3139 Accuracy2: 75.59%
Loss: 1.4461 Accuracy: 62.13%
Loss: 1.4461 Accuracy2: 75.31%
Loss: 1.3845 Accuracy: 62.17%
Loss: 1.3845 Accuracy2: 75.34%
Loss: 1.2780 Accuracy: 62.37%
Loss: 1.2780 Accuracy2: 75.38%
Loss: 1.3836 Accuracy: 62.07%
Loss: 1.3836 Accuracy2: 75.12%
Loss: 1.2114 Accuracy: 62.53%
Loss: 1.2114 Accuracy2: 75.47%
Loss: 1.1772 Accuracy: 62.77%
Loss: 1.1772 Accuracy2: 75.47%
Loss: 1.3592 Accuracy: 63.12%
Loss: 1.3592 Accuracy2: 75.60%
Loss: 1.3120 Accuracy: 63.08%
Loss: 1.3120 Accuracy2: 75.60%
Loss: 1.4866 Accuracy: 62.95%
Loss: 1.4866 Accuracy2: 75.46%
Loss: 1.1726 Accuracy: 63.17%
Loss: 1.1726 Accuracy2: 75.71%
Loss: 1.4586 Accuracy: 63.26%
Loss: 1.4586 Accuracy2: 75.81%
Loss: 1.5372 Accuracy: 63.00%
Loss: 1.5372 Accu

In [ ]:
# 2-2. MNIST 데이터를 4장씩 아래의 형태로 묶어서 분류할 수 있는 모델을 만들고 학습 및 테스트를 할 수 있는 코드를 제출하세요.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torchsummary import summary
import torch.nn.functional as F

normalize = transforms.Normalize(mean=[0.485],
                                 std=[0.229])

# 데이터 전처리 및 데이터 로드
transform = transforms.Compose([
    transforms.ToTensor(),
    normalize
])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# CIFAR-10 데이터셋 로드
cifar_train = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)

class CIFAR10Dataset(Dataset):
    def __init__(self, original_dataset):
        self.original_dataset = original_dataset

    def __len__(self):
        return len(self.original_dataset) // 4  # 4개씩 묶어서 사용

    def __getitem__(self, idx):
        img1, label1 = self.original_dataset[idx * 4]
        img2, label2 = self.original_dataset[idx * 4 + 1]
        img3, label3 = self.original_dataset[idx * 4 + 2]
        img4, label4 = self.original_dataset[idx * 4 + 3]

        concatenated_img1 = torch.cat([img1, img2], dim=2)
        concatenated_img2 = torch.cat([img3, img4], dim=2)
        concatenated_img = torch.cat([concatenated_img1, concatenated_img2], dim=1)
        concatenated_lbl = torch.tensor([label1, label2, label3, label4])

        return concatenated_img, concatenated_lbl

concatenated_train_data = CIFAR10Dataset(cifar_train)
# 이어붙인 데이터셋으로 DataLoader 생성
batch_size = 64

train_loader = DataLoader(concatenated_train_data, batch_size=batch_size, shuffle=True)

class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.conv3 = nn.Conv2d(64, 128, 3)
        self.conv4 = nn.Conv2d(128, 256, 3)
        self.drop1 = nn.Dropout()
        self.fc1 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)

        x = F.relu(self.conv2(x))
        x = self.pool(x)

        x = F.relu(self.conv3(x))
        x = self.pool(x)

        x = F.relu(self.conv4(x))
        x = self.pool(x)

        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.drop1(x)
        x = self.fc3(x)
        return x

# 모델, 손실 함수, 최적화 함수 정의
model = SimpleCNN().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
summary(model, input_size=(1, 56, 56))

# 훈련
epochs = 50
for epoch in range(epochs):
    for imgs, labels in train_loader:
        optimizer.zero_grad()
        imgs = imgs.to(device)
        outputs = model(imgs)
        new_labels = [[0.0 for tt in range(10)] for t in range(len(labels))] # 1 0 0 1 0 0 1 1 0 0 형태로 만들어주기 위한 배열
        for t in range(len(labels)):
          for j in range(len(labels[t])):
            new_labels[t][labels[t][j]] = 1.0
        labels = torch.tensor(new_labels)
        labels = labels.to(device)
        loss = criterion(outputs, labels) #loss 계산(BCEWithLogitsLoss)
        loss.backward()
        optimizer.step()
        pred_soft = torch.sigmoid(outputs)
        pred_binary = pred_soft.gt(0.5).type(outputs.type())
        total += (labels == labels).sum().item()
        correct += (pred_binary == labels).sum().item()

    print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}, Accuracy: {correct / total * 100:.2f}%') # Loss, Accuracy 출력

print('훈련 완료')

100%|██████████| 9912422/9912422 [00:00<00:00, 360596643.84it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 42444181.44it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 141948880.29it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 19680298.31it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 54, 54]             320
         MaxPool2d-2           [-1, 32, 27, 27]               0
            Conv2d-3           [-1, 64, 25, 25]          18,496
         MaxPool2d-4           [-1, 64, 12, 12]               0
            Conv2d-5          [-1, 128, 10, 10]          73,856
         MaxPool2d-6            [-1, 128, 5, 5]               0
            Conv2d-7            [-1, 256, 3, 3]         295,168
         MaxPool2d-8            [-1, 256, 1, 1]               0
            Linear-9                  [-1, 128]          32,896
          Dropout-10                  [-1, 128]               0
           Linear-11                   [-1, 10]           1,290
Total params: 422,026
Trainable params: 422,026
Non-trainable params: 0
--------------------

In [ ]:
# 테스트
model.eval().to(device)
correct = 0
total = 0
correct2 = 0
total2 = 0

with torch.no_grad():
    for imgs, labels in train_loader:
        imgs = imgs.to(device)
        outputs = model(imgs)
        pred_soft = torch.sigmoid(outputs)
        pred_binary = pred_soft.gt(0.5).type(outputs.type())
        new_labels = [[0.0 for tt in range(10)] for t in range(len(labels))]
        for t in range(len(labels)):
          for j in range(len(labels[t])):
            new_labels[t][labels[t][j]] = 1.0
        labels = torch.tensor(new_labels)
        labels = labels.to(device)
        loss = criterion(outputs, labels)
        total2 += (labels == labels).sum().item()
        correct2 += (pred_binary == labels).sum().item()
        for t in range(len(labels)):
          for j in range(len(labels[t])):
            if (labels[t][j] == 1.0):
              total += 1
              if (pred_binary[t][j] == 1.0):
                correct += 1
        print(f'Loss: {loss.item():.4f} Accuracy: {(correct / total * 100):.2f}%') # Label이 1일 때, 추정값도 1인 개수 비율
        print(f'Loss: {loss.item():.4f} Accuracy2: {(correct2 / total2 * 100):.2f}%') # 추정값 중 얼마나 맞았는지에 대한 비율 (Label과 추정값이 얼마나 같은지에 대한 비율)

accuracy = correct / total
accuracy2 = correct2 / total2
print(f'Final Test Accuracy: {accuracy * 100:.2f}%') # Label이 1일 때, 추정값도 1인 개수 비율
print(f'Final Test Accuracy2: {accuracy2 * 100:.2f}%') # 추정값 중 얼마나 맞았는지에 대한 비율 (Label과 추정값이 얼마나 같은지에 대한 비율)

Loss: 0.0286 Accuracy: 99.09%
Loss: 0.0286 Accuracy2: 99.69%
Loss: 0.0241 Accuracy: 98.86%
Loss: 0.0241 Accuracy2: 99.53%
Loss: 0.0127 Accuracy: 99.10%
Loss: 0.0127 Accuracy2: 99.58%
Loss: 0.0029 Accuracy: 99.32%
Loss: 0.0029 Accuracy2: 99.65%
Loss: 0.0027 Accuracy: 99.36%
Loss: 0.0027 Accuracy2: 99.69%
Loss: 0.0061 Accuracy: 99.39%
Loss: 0.0061 Accuracy2: 99.69%
Loss: 0.0001 Accuracy: 99.48%
Loss: 0.0001 Accuracy2: 99.73%
Loss: 0.0078 Accuracy: 99.55%
Loss: 0.0078 Accuracy2: 99.75%
Loss: 0.0086 Accuracy: 99.54%
Loss: 0.0086 Accuracy2: 99.76%
Loss: 0.0026 Accuracy: 99.59%
Loss: 0.0026 Accuracy2: 99.77%
Loss: 0.0017 Accuracy: 99.59%
Loss: 0.0017 Accuracy2: 99.77%
Loss: 0.0003 Accuracy: 99.62%
Loss: 0.0003 Accuracy2: 99.79%
Loss: 0.0005 Accuracy: 99.65%
Loss: 0.0005 Accuracy2: 99.81%
Loss: 0.0080 Accuracy: 99.64%
Loss: 0.0080 Accuracy2: 99.80%
Loss: 0.0011 Accuracy: 99.67%
Loss: 0.0011 Accuracy2: 99.81%
Loss: 0.0024 Accuracy: 99.69%
Loss: 0.0024 Accuracy2: 99.81%
Loss: 0.0013 Accuracy: 9

In [ ]:
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

concatenated_test_data = CIFAR10Dataset(testset)
# 이어붙인 데이터셋으로 DataLoader 생성
batch_size = 64
test_loader = DataLoader(concatenated_test_data, batch_size=batch_size, shuffle=True)

correct = 0
total = 0
correct2 = 0
total2 = 0

with torch.no_grad():
    for imgs, labels in test_loader:
        imgs = imgs.to(device)
        outputs = model(imgs)
        pred_soft = torch.sigmoid(outputs)
        pred_binary = pred_soft.gt(0.5).type(outputs.type())
        new_labels = [[0.0 for tt in range(10)] for t in range(len(labels))]
        for t in range(len(labels)):
          for j in range(len(labels[t])):
            new_labels[t][labels[t][j]] = 1.0
        labels = torch.tensor(new_labels)
        labels = labels.to(device)
        loss = criterion(outputs, labels)
        total2 += (labels == labels).sum().item()
        correct2 += (pred_binary == labels).sum().item()
        for t in range(len(labels)):
          for j in range(len(labels[t])):
            if (labels[t][j] == 1.0):
              total += 1
              if (pred_binary[t][j] == 1.0):
                correct += 1
        print(f'Loss: {loss.item():.4f} Accuracy: {(correct / total * 100):.2f}%') # Label이 1일 때, 추정값도 1인 개수 비율
        print(f'Loss: {loss.item():.4f} Accuracy2: {(correct2 / total2 * 100):.2f}%') # 추정값 중 얼마나 맞았는지에 대한 비율 (Label과 추정값이 얼마나 같은지에 대한 비율)

accuracy = correct / total
accuracy2 = correct2 / total2
print(f'Final Test Accuracy: {accuracy * 100:.2f}%') # Label이 1일 때, 추정값도 1인 개수 비율
print(f'Final Test Accuracy2: {accuracy2 * 100:.2f}%') # 추정값 중 얼마나 맞았는지에 대한 비율 (Label과 추정값이 얼마나 같은지에 대한 비율)

Loss: 0.4172 Accuracy: 93.06%
Loss: 0.4172 Accuracy2: 96.41%
Loss: 0.3131 Accuracy: 94.12%
Loss: 0.3131 Accuracy2: 96.95%
Loss: 0.0435 Accuracy: 95.53%
Loss: 0.0435 Accuracy2: 97.50%
Loss: 0.1098 Accuracy: 95.51%
Loss: 0.1098 Accuracy2: 97.58%
Loss: 0.3032 Accuracy: 95.16%
Loss: 0.3032 Accuracy2: 97.38%
Loss: 0.2119 Accuracy: 95.25%
Loss: 0.2119 Accuracy2: 97.40%
Loss: 0.2616 Accuracy: 94.99%
Loss: 0.2616 Accuracy2: 97.23%
Loss: 0.2568 Accuracy: 94.84%
Loss: 0.2568 Accuracy2: 97.21%
Loss: 0.2344 Accuracy: 95.01%
Loss: 0.2344 Accuracy2: 97.22%
Loss: 0.4210 Accuracy: 94.83%
Loss: 0.4210 Accuracy2: 97.06%
Loss: 0.3365 Accuracy: 94.82%
Loss: 0.3365 Accuracy2: 97.09%
Loss: 0.2903 Accuracy: 94.89%
Loss: 0.2903 Accuracy2: 97.10%
Loss: 0.2222 Accuracy: 94.85%
Loss: 0.2222 Accuracy2: 97.09%
Loss: 0.2175 Accuracy: 94.86%
Loss: 0.2175 Accuracy2: 97.14%
Loss: 0.1513 Accuracy: 94.97%
Loss: 0.1513 Accuracy2: 97.21%
Loss: 0.3398 Accuracy: 94.98%
Loss: 0.3398 Accuracy2: 97.16%
Loss: 0.3007 Accuracy: 9